In [1]:
import sys
sys.path.append('/cluster/sj1/bb_opt/scripts')

In [2]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collect_stats as cs

%matplotlib inline

In [3]:
import collect_stats as cs

In [4]:
exp_folder = '/cluster/sj1/bb_opt/experiments/imdbwiki'
batches = [20, 50, 200]
num_acks = 50

In [5]:
filenames = ['wiki']

In [6]:
def sigmoid(x, exp=np.exp):
  return 1.0 / (1.0 + exp(-x))

In [7]:
arrs = {}

In [205]:
to_read = {
    #'ensemble8/o_max_std_2k_reg_maxvar_g00020510_': [None, 'e8_2k_maxvar_g00020510'],
    #'ensemble8/o_max_std_2k_reg_maxinvar_g00020510_': [None, 'e8_2k_maxinvar_g00020510'],
    #'ensemble8/o_max_std_2k_reg_maxvar_g0005102040_': [None, 'e8_2k_maxvar_g0005102040'],
    #'ensemble8/o_max_std_2k_reg_maxvar_g0002051020_': [None, 'MOD'],
    #'ensemble8/o_max_std_2k_reg_maxinvar_g0002051020_': [None, 'MTD'],
    #'ensemble8/o_max_std_2k_reg_maxinoutvar_g0002051020_': [None, 'e8_2k_maxinoutvar_g0002051020'],
    #'ensemble8/o_max_std_2k_reg_inverse_density_maxvar_g0002051020_': [None, 'MODD on male'],
    #'ensemble8/o_max_std_re_2k_reg_inverse_ood_maxvar_g0002051020_': [None, 'MODD on female'],
    #'ensemble8/o_max_std_2k_reg_ood_maxvar_g0002051020_': [None, 'MOD on female'],
    #'ensemble9/o_max_std_2k_reg_inverse_density_maxvar_g0002051020_': [None, 'e9_2k_inverse_maxvar_g0002051020'],
    #'ensemble9/o_max_std_2k_reg_maxvar_g0002051020_': [None, 'e9_2k_maxvar_g0002051020'],
    #'ensemble9/o_max_std_2k_reg_inverse_density_bbfc_maxvar_g0002051020_': [None, 'e9_2k_inverse_bbfc_maxvar_g0002051020'],
    #'ensemble9/o_max_std_2k_reg_inverse_density_pomfc_maxvar_g0002051020_': [None, 'e9_2k_inverse_pomfc_maxvar_g0002051020']
    #'ensemble9/o_max_std_2k_reg_inverse_density_pomfc_maxvar_g0002051020_': [None, 'pomfc'],
    #'ensemble13/o_max_std_2k_reg_ood_maxvar_g0002051020_': [None, 'MOD on OOD'],
    #'ensemble14/o_max_std_2k_reg_normal_': [None, 'Normal'],
    #'ensemble14/o_max_std_2k_reg_ood_maxvar_g0002051020_': [None, 'ood'],
    #'ensemble14/o_max_std_2k_reg_ood_inverse_maxvar_g0002051020_': [None, 'ood_inverse'],
    'ensemble14/o_max_std_2k_reg_uinput_maxvar_g0002051020_': [None, 'uinput'],
    'ensemble14/o_max_std_2k_reg_uinput_inverse_maxvar_g0002051020_': [None, 'uinput_inverse'],
    'ensemble14/o_max_std_2k_reg_uinput_langevin_maxvar_g0002051020_': [None, 'uinput_langevin_inverse'],
    #'ensemble14/o_max_std_2k_reg_ood_embinverse_maxvar_g0002051020_': [None, 'ood_embinverse'],
    #'ensemble14/o_max_std_2k_reg_indist_embinverse_maxvar_g0002051020_': [None, 'indist_embinverse'],
    #'ensemble14/o_max_std_2k_reg_indist_maxvar_g0002051020_': [None, 'indist'],
    #'ensemble14/o_max_std_2k_reg_indist_inverse_maxvar_g0002051020_': [None, 'indist_inverse'],
    #'ensemble13/o_max_std_2k_reg_inverse_ood_maxvar_g0002051020_': [None, 'MODD on OOD'],
    #'ensemble13/o_max_std_2k_reg_inverse_uniput_maxvar_g0002051020_': [None, 'MODD on uinput'],
    #'ensemble13/o_max_std_2k_reg_uniput_maxvar_g0002051020_': [None, 'MOD on uinput'],
    #'ensemble13/o_max_std_2k_reg_inverse_bbfc_maxvar_g0002051020_': [None, 'MODD on bbfc'],
    #'ensemble13/o_max_std_2k_reg_inverse_ufc_maxvar_g0002051020_': [None, 'MODD on ufc'],
}


In [206]:
for k in to_read:
    arrs[k] = to_read[k]
for experiment in to_read:
    print('reading', experiment)
    arrs[experiment][0] = cs.get_data(exp_folder, experiment, batches, num_samples=20)

reading ensemble14/o_max_std_2k_reg_uinput_maxvar_g0002051020_
reading sample 1
reading sample 2
reading sample 3
reading sample 4
reading sample 5
reading sample 6
reading sample 7
reading sample 8
reading sample 9
reading sample 10
reading sample 11
reading sample 12
reading sample 13
reading sample 14
reading sample 15
reading sample 16
reading sample 17
reading sample 18
reading sample 19
reading sample 20
reading ensemble14/o_max_std_2k_reg_uinput_inverse_maxvar_g0002051020_
reading sample 1
reading sample 2
reading sample 3
reading sample 4
reading sample 5
reading sample 6
reading sample 7
reading sample 8
reading sample 9
reading sample 10
reading sample 11
reading sample 12
reading sample 13
reading sample 14
reading sample 15
reading sample 16
reading sample 17
reading sample 18
reading sample 19
reading sample 20
reading ensemble14/o_max_std_2k_reg_uinput_langevin_maxvar_g0002051020_
reading sample 1
reading sample 2
reading sample 3
reading sample 4
reading sample 5
reading

In [207]:
to_eval = [
    'ensemble14/o_max_std_2k_reg_ood_maxvar_g0002051020_',
    'ensemble14/o_max_std_2k_reg_ood_inverse_maxvar_g0002051020_',
    'ensemble14/o_max_std_2k_reg_ood_embinverse_maxvar_g0002051020_',
    'ensemble14/o_max_std_2k_reg_indist_maxvar_g0002051020_',
    'ensemble14/o_max_std_2k_reg_indist_inverse_maxvar_g0002051020_',
    'ensemble14/o_max_std_2k_reg_indist_embinverse_maxvar_g0002051020_',
    'ensemble14/o_max_std_2k_reg_uinput_maxvar_g0002051020_',
    'ensemble14/o_max_std_2k_reg_uinput_inverse_maxvar_g0002051020_',
    'ensemble14/o_max_std_2k_reg_uinput_langevin_maxvar_g0002051020_',
]

In [147]:
test_fn = lambda bs, prop: lambda x, filename : x[bs + 'test_pred_stats'][prop]
bs_test_rmse_fn = lambda x, filename : x['test_pred_stats']['baseline_rmse']
ood_fn = lambda bs, prop: lambda x, filename : x[bs + 'ood_pred_stats'][prop]
bs_ood_rmse_fn = lambda x, filename : x['test_pred_stats']['baseline_rmse']
test_log_fn = lambda prop, prop2: lambda x, filename : x['logging'][1][prop][prop2]

In [210]:
dist = "test"
dist = "ood"
prop = "log_prob"
#prop = "rmse"
#prop = "pred_std"
#prop = 'rmse_std_corr'
bs = "zero_gamma_"
#bs = "invar_gamma_"

#fn = [globals()[dist+"_" + prop + "_fn"](""),globals()[dist + "_" + prop + "_fn"](bs)]
fn = [globals()[dist+"_fn"]("", prop), globals()[dist+"_fn"](bs, prop)]
cs.prop_test2(50, 
             filenames,
             fn, 
             arrs,
             to_eval[-1],
             0,
             pval_threshold=1,
             paired_test=True,
            )

comparing <lambda> <lambda>
combined pval: nan vs nan
count: 0/0


In [23]:
cs.prop_test(50, 
             filenames,
             ood_fn("", "log_prob"), 
             arrs, 
             [to_eval[0], to_eval[1]],
             0,
             pval_threshold=1, 
             paired_test=False,
            )

comparing Normal ood
wiki 0.43066987180057853 -0.1295899584889412 -0.15854592472314835 0 	 (std: 0.06006 0.03350, #n: 5 5)
combined pval: 0.43067 vs nan
count: 0/1


In [ ]:
cs.plot_data_vs_ack_iter(
    50,
    filenames,
    'avg_seeds',
    'rmse',
    test_rmse_fn,
    20,
    arrs,
    to_eval,
    legend_loc=3,
)

In [ ]:
cs.plot_data_vs_ack_iter(
    20,
    filenames,
    'avg_seeds',
    'log_prob',
    lambda x, filename: x['best_gamma'],
    30,
    arrs,
    to_eval,
    legend_loc=3,
)

In [ ]:
cs.plot_data_vs_ack_iter(
    50,
    filenames,
    'avg_seeds',
    'nll',
    lambda x, filename : x['logging'][1]['best']['nll'],
    20,
    arrs,
    to_eval,
    legend_loc=3,
)

In [ ]:
cs.plot_data_vs_ack_iter(
    200,
    filenames,
    'avg_seeds',
    'rmse',
    test_rmse_fn,
    20,
    arrs,
    to_eval,
    legend_loc=3,
)